In [1]:
import numpy as np
from dynamicSceneGenerator import DynamicSceneGenerator
from visualize import visualize_dynamic_scene, visualize_camera_pose_in_dsg, visualize_projections, visualize_bounding_boxes
from MODSIM import create_and_place_simple_legacy_camera, project_all_points_from_json, project_all_points, create_all_bbs_from_json, create_dynamic_scene_with_random_tracks 
from utils import find_path_to_next_simulation

# Might need to pip install json, json_stream, as they are not default libraries
import json
import os

In [2]:
import numpy as np
from dynamicSceneGenerator import DynamicSceneGenerator
from visualize import visualize, visualize_camera_gif, visualize_projections, visualize_bounding_boxes
from datatypes.virtualCamera import VirtualCamera
from datatypes.boundingBox import BoundingBoxfrom datatypes.track import Track


# Might need to pip install json, json_stream, as they are not default libraries
import json
import os

from datatypes.vessel import Vessel
# Find folder to save simultation trial to
simulation_folder = find_path_to_next_simulation()

In [3]:
# Generate dynamic scene with random tracks
dsg = create_dynamic_scene_with_random_tracks(6, writeToJson=True, path=simulation_folder)

In [3]:
def create_and_place_simple_legacy_camera(largest_radius, path_centre): # This should maybe get a better name, and should be moved somewhere?
    '''
    Function for placing a Simple legacy photo camera in the dynamic scene
    '''
    # NB! Make sure everything is in meters
    focal_length = 50*10**-3
    image_bounds = (3600, 2400) # Pixels (x,y)
    film_size = (36*10**-3, 24*10**-3)
    px = film_size[0]/image_bounds[0]
    py = film_size[1]/image_bounds[1]
    principal_point = (image_bounds[0]/2,image_bounds[1]/2)
    width_of_sensor = 36*10**-3 # Width of sensor
    camera_height = 60 # metre

    position_WRF = calculate_camera_position_WRF(focal_length, width_of_sensor, largest_radius, path_centre, camera_height)
    roll = 0
    yaw = np.pi
    pitch = calculate_camera_pitch(position_WRF)
    
    camera = VirtualCamera(position_WRF, roll, yaw, pitch, focal_length, px, py, principal_point, image_bounds)

    return camera# Visualize dynamic scene
vessels = dsg.get_vessels()
visualize_dynamic_scene(vessels, folder_path=simulation_folder)

In [5]:
# Create simple legacy camera and place it in the dynamic scene
camera = create_and_place_simple_legacy_camera(dsg.get_larges_radius(), dsg.get_path_centre())

# Visualize the camera position
visualize_camera_pose_in_dsg(camera, vessels, folder_path=simulation_folder)

In [6]:
# NOT read from JSON
# all_projected_points = project_all_points(camera, vessels)

# Read from Json
all_projected_points = project_all_points_from_json(camera, simulation_folder, writeToJson=True)

# Visualize projections
visualize_projections(all_projected_points, camera.image_bounds, folder_path=simulation_folder)

In [7]:
# Read projected points from JSON and create bounding boxes
all_bbs = create_all_bbs_from_json(simulation_folder, writeToJson=True)

In [9]:
def dict_to_json(path, dict):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, 'w+') as f:
        json.dump(dict, f, indent = 4)


In [17]:
def tracks_to_json(vessels, path):
    # NB! Assumes all vessels have the same timestamp
    all_tracks = {key: {vessel.id: vessel.get_track_dict()[key] for vessel in vessels} for key in vessels[0].get_track_dict().keys()}
    filename = os.path.join(path, 'tracks.json')
    dict_to_json(filename, all_tracks)

In [18]:
def vessels_to_json(vessels, path):
    filename = os.path.join(path, 'vehicel_characteristics.json')
    vessel_dict = {vessel.id: {'air_draft_m': vessel.air_draft, 'beam_m': vessel.beam, 'length_m': vessel.length, 'label': vessel.label} for vessel in vessels}
    dict_to_json(filename, vessel_dict)

In [59]:
def projectedPoints_to_json(projected_points, vessels, path):
    # All projected points will probably be on a different format. Did this to get a file.
    projected_points_dict = {key: {vessels[i].id: {x: {'x': projected_points[key][i][x][0], 'y': projected_points[key][i][x][1], 'z': projected_points[key][i][x][2]} for x in range(len(projected_points[key][i]))} for i in range(len(projected_points[key]))} for key in projected_points.keys()}
    filename = os.path.join(path, 'projectedPoints.json')
    dict_to_json(filename, projected_points_dict)

In [79]:
def project_all_points(camera, vessels):
    all_projected_points = {}
    for t in vessels[0].get_track().get_time_stamps():
        points = [vessel.calculate_3D_cornerpoints(t) for vessel in vessels]
        projected_points = [camera.project_points(vessel_points) for vessel_points in points]
        all_projected_points[t] = projected_points
    return all_projected_points

# Where should we save the projected points? A separate class, in the dynamic scene?
# And also the functions in this file should be in a separate class?

In [5]:
def create_bound_boxes(projected_points):
    bbs = []
    for vessel in projected_points:
        if vessel.size > 0:
            vessel_x = np.array([point.image_coordinate[0] for point in vessel])
            vessel_y = np.array([point.image_coordinate[1] for point in vessel])
            vessel_depth = np.array([point.depth for point in vessel])
            max_x = np.max(vessel_x)
            min_x = np.min(vessel_x)
            max_y = np.max(vessel_y)
            min_y = np.min(vessel_y)
            width = max_x-min_x
            height = max_y - min_y
            centre = [min_x+width/2, min_y+height/2]
            depth = np.average(vessel_depth) #OBS which depth should we use
            bounding_box = BoundingBox(None, centre, width, height, depth)
            # Make sure these are in the correct order as in master thesis
            bbs.append(bounding_box)
    return bbs


In [89]:
def project_all_points_from_json(camera, folder_path, writeTojson=True):
    vessel_path = os.path.join(folder_path, 'vehicel_characteristics.json')
    track_path = os.path.join(folder_path, 'tracks.json')
    vessels = json_to_vessels(vessel_path)
    tracks = json_to_tracks(track_path)
    for vesselID, track in tracks.items(): 
        vessels[vesselID].set_track(track)
    vessel_list = list(vessels.values())
    projected_points = project_all_points(camera, vessel_list)
    if writeTojson:
        projectedPoints_to_json(projected_points, vessel_list, folder_path)
    return projected_points

In [6]:
def handle_covered_bbs(bounding_boxes):
    bbs = []
    sorted_bbs = sorted(bounding_boxes, key=lambda bb: bb.depth, reverse=True)
    for i in range(len(bounding_boxes)-1):
        bb = sorted_bbs[i]
        fully_covered = False
        covering_bbs = []
        for j in range(i+1, len(bounding_boxes)):
            if bb.check_fully_covered(sorted_bbs[j]):
                fully_covered = True
            elif bb.check_overlap(sorted_bbs[j]):
                covering_bbs.append(sorted_bbs[j])
        if len(covering_bbs) > 0:
            bb.update_bb_if_covered(covering_bbs)
        if not fully_covered:
            bbs.append(bb)
    bbs.append(sorted_bbs[-1])
    return bbs

In [7]:
def create_all_bbs(all_projected_points):
    all_bbs = {}
    for t in all_projected_points.keys():
        temp=create_bound_boxes(all_projected_points[t])
        all_bbs[t] = handle_covered_bbs(temp)
    return all_bbs

In [8]:
# Generate dynamic scene with random tracks
dsg = DynamicSceneGenerator()
dsg.set_random_vessels(6)
dsg.generate_random_tracks()
vessels = dsg.get_vessels()
visualize(vessels)

In [87]:
def find_path_to_next_simulation():
    arr = os.listdir('./simulations/')
    timestamp = 0 if not arr else int(arr[-1])+1
    filename = f'./simulations/{timestamp}/'
    return filename

In [9]:
# Generate dynamic scene with random tracks
dsg = DynamicSceneGenerator()
dsg.set_random_vessels(6)
dsg.generate_random_tracks()
vessels = dsg.get_vessels()
path = find_path_to_next_simulation()
tracks_to_json(vessels, path)
vessels_to_json(vessels, path)
# visualize(vessels)

In [57]:
camera = create_and_place_simple_legacy_camera(dsg.get_larges_radius(), dsg.get_path_centre())
# visualize_camera_gif(camera, vessels)

In [10]:
# all_projected_points = project_all_points(camera, vessels)
# visualize_projections(all_projected_points, camera.image_bounds)

vessel_path = os.path.join(path, 'vehicel_characteristics.json')
track_path = os.path.join(path, 'tracks.json')
folder_path = path
all_projected_points = project_all_points_from_json(camera, folder_path, writeTojson=True)

In [11]:
# All projected points will probably be on a different format. Did this to get a file.
projected_points_dict = {key: {vessels[i].id: {x: {'x': all_projected_points[key][i][x][0], 'y': all_projected_points[key][i][x][1], 'z': all_projected_points[key][i][x][2]} for x in range(len(all_projected_points[key][i]))} for i in range(len(all_projected_points[key]))} for key in all_projected_points.keys()}
filename = f'./simulations/0/projectedPoints.json'
dict_to_json(filename, projected_points_dict)

In [16]:
all_bbs = create_all_bbs(projected_points_dict)
filename = f'./simulations/0/boundingBoxes.json'
dict_to_json(filename, all_bbs)

# With points in BBs
visualize_bounding_boxes(all_bbs, camera.image_bounds, projected_points=all_projected_points, show_projected_points=False)
#visualize_bounding_boxes(all_bbs, camera.image_bounds, projected_points=projected_points_dict, show_projected_points=True)
# Visualize BBS
# With points in BBs
# FIX THIS WHEN RIGHT PROJECTED POINTS ARE HERE
# visualize_bounding_boxes(all_bbs, camera.image_bounds, projected_points=all_projected_points, show_projected_points=True, folder_path=simulation_folder)
# Without
visualize_bounding_boxes(all_bbs, camera.image_bounds, folder_path=simulation_folder)